In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# from __future__ import (absolute_import, division, print_function,
#                         unicode_literals)
from collections import defaultdict
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

SyntaxError: from __future__ imports must occur at the beginning of the file (<ipython-input-3-ffd0e97c3391>, line 10)

In [ ]:
import glob

path = r'../input/test-recsys/sbermarket_tab_2_1' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, usecols=['user_id','product_id','quantity'])
    li.append(df)

frame1 = pd.concat(li, axis=0, ignore_index=True)

path = r'../input/test-recsys/sbermarket_tab_2_2' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, usecols=['user_id','product_id','quantity'])
    li.append(df)

frame2 = pd.concat(li, axis=0, ignore_index=True)

path = r'../input/test-recsys/sbermarket_tab_2_3' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, usecols=['user_id','product_id','quantity'])
    li.append(df)

frame3 = pd.concat(li, axis=0, ignore_index=True)

# path = r'../input/test-recsys/sbermarket_tab_2_4' # use your path
# all_files = glob.glob(path + "/*.csv")

# li = []

# for filename in all_files:
#     df = pd.read_csv(filename, usecols=['user_id','product_id','quantity'])
#     li.append(df)

# frame4 = pd.concat(li, axis=0, ignore_index=True)

# path = r'../input/test-recsys/sbermarket_tab_2_5' # use your path
# all_files = glob.glob(path + "/*.csv")

# li = []

# for filename in all_files:
#     df = pd.read_csv(filename, usecols=['user_id','product_id','quantity'])
#     li.append(df)

# frame5 = pd.concat(li, axis=0, ignore_index=True)

# path = r'../input/test-recsys/sbermarket_tab_2_6' # use your path
# all_files = glob.glob(path + "/*.csv")

# li = []

# for filename in all_files:
#     df = pd.read_csv(filename, usecols=['user_id','product_id','quantity'])
#     li.append(df)

# frame6 = pd.concat(li, axis=0, ignore_index=True)


In [ ]:
frame=pd.concat([frame1,frame2,frame3], axis=0, ignore_index=True)

In [ ]:
frame

In [ ]:
neworder = ['user_id','product_id','quantity']
df=frame.reindex(columns=neworder)

In [ ]:
del frame
del frame1
del frame2
del frame3
#del frame4
#del frame5
#del frame6

In [ ]:
df

In [4]:
!pip install scikit-surprise

In [5]:
reader=Reader(rating_scale=(1, 5))

NameError: name 'Reader' is not defined

In [ ]:
data = Dataset.load_from_df(df, reader)

In [ ]:
data

In [ ]:
train_set, test_set = train_test_split(data, test_size=0.2, shuffle=True)

In [ ]:
svd_algo=SVD()

In [ ]:
svd_algo.fit(train_set)

In [ ]:
predictions=svd_algo.test(test_set)

In [ ]:
predictions

In [ ]:
def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.
    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.
    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [ ]:
top_pred=get_top_n(predictions, 50)

In [ ]:
top_pred

In [ ]:
submission=pd.read_csv('/kaggle/input/test-recsys/sample_submission.csv')

In [ ]:
submission

In [ ]:
from IPython.display import HTML
import pandas as pd
import numpy as np
import base64

# function that takes in a dataframe and creates a text link to  
# download it (will only work for files < 2MB or so)
def create_download_link(df, title = "Download CSV file", filename = "data.csv"):  
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

# create a random sample dataframe
df = pd.DataFrame(np.random.randn(50, 4), columns=list('ABCD'))

# create a link to download the dataframe
create_download_link(df)